--- 
# Cell tracking and interactions

This is a program that is designed to take as an input a parent directory containing many drift-corrected stack images from the Incucyte microscope, and examine the interaction between two different cell lines. 

Process: 
1. User inputs the time between frames and the image resolution for the microscope. 
2. User selects the parent folder containing the data to analyse using a file-dialogue box. 
3. For all image folders (plates and wells)  
    a. Read in the image data for all colour channels.   
    b. Segment the objects within the time-stack images using cellpose.   
    c. Sort the segmentations into different cell types based on the output of the cellpose segmentation.  
    d. Use btrack to track the cells in the time-lapse.   
    e. Filter the tracks to only include tracks covering 90% of the time-lapse. 
    f. Monitor interactions between the two different cell types.   
    g. Save the results of the image analysis

---
### Import dependancies

In [ ]:
import tifffile as tf 
import numpy as np 
import napari 
import pandas as pd

import tkinter as tk 
from tkinter import filedialog

import os
import tqdm

from cellpose import core, models, io, metrics
# from cellpose.contrib import openvino_utils

import btrack
import skimage
import scipy

import matplotlib.pyplot as plt

----
### User Inputted meta Data

In [ ]:
time_between_frames = 2 # in minutes

image_resolution = 1.24 # micron, for 10x on incucyte S3

---
### Grab the image files

In [ ]:
# Generate a file dialogue box to select folder 
# containing the images to analyse. 
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw() # Stops a second window opening
images_folder = filedialog.askdirectory(title = 'Select image Folder')

In [ ]:
print(images_folder)#

In [ ]:
def find_image_folders(directory):
    '''Enter the folder directory and select only the sub-folders that
    contain the images for analysis. '''
    
    # Initalise. 
    found_directories = []
    
    # Loop through all folders and files in the directory 
    for roots, directories, files in os.walk(directory):
        # For all subfolders. 
        for j in range( len(directories)):
            # Save the sub-folder paths with the images within them. 
            if directories[j].find('Colour') != -1: 
                found_directories.append(roots + '/' + directories[j])

    return(found_directories)

---
### Segment Green cells. 

In [ ]:
def run_cellpose(segmentation_model, stack_image, diam):
    '''A method for performing cellpose segmentation, which should be 
    agnostic of the inputs.'''
    
    # Load the cellpose model.
    model = models.CellposeModel(gpu=True, model_type=segmentation_model)
    # model = openvino_utils.to_openvino(model)
    
    # initialise variables. 
    cellpose_masks = []
    object_number = []

    # Cellpose segmentation of all frames in the analysis. 
    for i in tqdm.tqdm(range( stack_image.shape[0] )):  
        # Cellpose segmentation of the filtered data. 
        masks, flows, styles = model.eval(stack_image[i, ...], channels=[0, 0], diameter = diam)
        # Append masks to a stack image. 
        cellpose_masks.append(masks)
        # Save the numebr of objects found per-frame. 
        object_number.append(np.max(masks))
    
    return(np.array(cellpose_masks), object_number)

In [ ]:
def mean_thresholding(image):
    '''Use the mean threshold in skimage to reduce the 
    background on the fluorescence images. '''
    
    # Initalise
    threshold_stack = []
    
    # Cellpose segmentation of all frames in the analysis. 
    for i in range( image.shape[0] ): 
        # Get the mean filter value from the image. 
        mean_filter_val = skimage.filters.threshold_mean(image[i, ...])
        
        # print(mean_filter_val)
        # Filter the image frame data using the 
        # mean filter. 
        mean_filter_frame = np.array(image[i, ...])
        mean_filter_frame[mean_filter_frame < mean_filter_val] = 0
        
        threshold_stack.append(mean_filter_frame)
    
    return(np.array(threshold_stack))

In [ ]:
def otzu_thresholding(image):
    '''Use the mean threshold in skimage to reduce the 
    background on the fluorescence images. '''
    
    # Initalise
    threshold_stack = []
    
    im_labels = []
    
    # Cellpose segmentation of all frames in the analysis. 
    for i in range( image.shape[0] ): 
        # Get the mean filter value from the image. 
        mean_filter_val = 4*skimage.filters.threshold_mean(image[i, ...])
        
        # Perform segmentation from the Otzu filter 
        thresh_im = image[i, ...] > mean_filter_val
        im_labels.append(skimage.morphology.label(thresh_im))
        
        # print(mean_filter_val)
        # Filter the image frame data using the 
        # mean filter. 
        mean_filter_frame = np.array(image[i, ...])
        mean_filter_frame[mean_filter_frame < mean_filter_val] = 0
        
        threshold_stack.append(mean_filter_frame)
    
    return(np.array(threshold_stack), np.array(im_labels))

In [ ]:
def filter_masks_by_overlapping_masks(phase, green, red):
    '''If the phase image masks overlap with the fluorescence image cell masks 
    then it is removed from the phase-masks used for analysis. '''

    # initalise
    macrophages = np.zeros(green.shape)
    gamma_deltas = np.zeros(green.shape)
    dead = np.zeros(green.shape)
    params = ['area', 'centroid']

    # For frames in image. 
    for i in range(phase.shape[0]):
        # initalise counters
        macro_num = 0
        gdt_num = 0
        dead_count = 0

        # Get phase frame
        phase_frame = phase[i, ...]
        # Get green frame
        green_frame = green[i, ...]
        # dead frame
        red_frame = red[i, ...]

        # Find the area and centroid position for all masks in frame [i]. 
        mask_props = pd.DataFrame(skimage.measure.regionprops_table(
                phase_frame, properties = params)) 

        for j in range( mask_props.shape[0]): 
            # Find mask value
            mask_num = phase[i, int(mask_props['centroid-0'].iloc[j]),  int(mask_props['centroid-1'].iloc[j])]
            # Find all mask indicies. 
            mask_indicies = np.where(phase_frame == mask_num)

            # Get the phase mask inprint on the green segmentation. 
            mask_on_green = green_frame[mask_indicies]
            # Find pixels greater than 0. 
            green_mask_positive = len(np.where(mask_on_green > 0)[0])

            # Get the phase mask inprint on the red segmentation. 
            mask_on_red = red_frame[mask_indicies]
            # Find pixels greater than 0. 
            red_mask_positive = len(np.where(mask_on_red > 0)[0])
            
            # If red has a 50% overlap or more with phase, 
            # Cell is considered dead. 
            if red_mask_positive > 0.5* len(mask_indicies[0]): 
                # Add 1 to cell count
                dead_count = dead_count + 1
                # Get a single frame.
                frame = np.array(phase_frame)
                # Convert not mask to zeros. 
                frame[frame != mask_num] = 0
                frame[frame == mask_num] = macro_num
                # Add mask to dead cell masks. 
                dead[i, ...] = dead[i, ...] + frame
            
            # If green mask has a 20% overlap with phase mask, 
            # Object considered a macrophage. 
            if green_mask_positive > 0.2* len(mask_indicies[0]): 
                # Add 1 to cell count
                macro_num = macro_num + 1
                # Get a single frame.
                frame = np.array(phase_frame)
                # Convert not mask to zeros. 
                frame[frame != mask_num] = 0
                frame[frame == mask_num] = macro_num
                # Add mask to macrophage masks. 
                macrophages[i, ...] = macrophages[i, ...] + frame

            else: 
                # Add 1 to cell count
                gdt_num = gdt_num + 1
                # Get a single frame.
                frame = np.array(phase_frame)
                # Convert not mask to zeros. 
                frame[frame != mask_num] = 0
                frame[frame == mask_num] = gdt_num
                # Add mask to GDT cell mask. 
                gamma_deltas[i, ...] = gamma_deltas[i, ...] + frame    

    return(gamma_deltas.astype(int), macrophages.astype(int), dead.astype(int))

In [ ]:
def filter_masks_by_size(phase_masks_stack):
    '''Filter the masks by the mean size of mask size. 
    To avoid deleteling lots of cells, I have set the filter to 
    remove cells larger than twice the mean cell size.'''
    
    # Initalise. 
    filter_by_size = np.zeros(phase_masks_stack.shape)
    params = ['area', 'centroid']

    # Count the number of cells in the mask stack. 
    cell_count = count_cell_masks(phase_masks_stack)
    
    # For all the frames. 
    for i in range(phase_masks_stack.shape[0]): 
        # Find all the cell mask data (area and position). 
        all_mask_data = pd.DataFrame(skimage.measure.regionprops_table(
            phase_masks_stack[i, ...], properties = params)) 
        
        if len(all_mask_data) == 0:
            continue
        else:
            # Get just the area. 
            all_mask_area = all_mask_data['area']
            # Get the mask area. 
            mean_mask_size = all_mask_area.mean()

            # For all cell masks. 
            for j in range(all_mask_area.shape[0]):
                # If the mask area is smaller than 1.3 times the mean area. 
                if all_mask_area[j] < 1.3 * mean_mask_size:
                    # Get the mask value.
                    mask_val = phase_masks_stack[i, int(all_mask_data['centroid-0'].iloc[j]), 
                                                  int(all_mask_data['centroid-1'].iloc[j])]

                    # Get the single frame. 
                    single_mask = np.array(phase_masks_stack[i, ...])
                    # Set everything not the mask in question to zero. 
                    single_mask[single_mask != mask_val] = 0
                    # Add single mask to mask stack. 
                    filter_by_size[i, ...] = filter_by_size[i, ...] + single_mask

    return(np.array(filter_by_size).astype(int))

---- 
### Track Cells

In [ ]:
def find_Btracks(image_masks, configer_file, loc, save_loc, im_name):
    '''A method to find the tracks from the cellpose masks
    generated from the image stack. 
    this method makes use of the trackpy package.'''
    
    FEATURES = [
      "area",
      "major_axis_length",
      "minor_axis_length",
       "eccentricity",
      "solidity",
                ]

    objects = btrack.utils.segmentation_to_objects(image_masks, properties=tuple(FEATURES))
    
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure_from_file(loc + configer_file)

        tracker.features = FEATURES
        
        # Add maximum cell tracking Distance
        tracker.max_search_radius = 15

        # append the objects to be tracked
        tracker.append(objects) 

        # set the volume (Z axis volume is set very large for 2D data)
        tracker.volume=((0, 2000), (0, 2000), (-1e5, 1e5))

        # track them (in interactive mode)
        tracker.track(step_size=100)

        # generate hypotheses and run the global optimizer
        tracker.optimize()

        # store the data in an HDF5 file
        tracker.export(save_loc + im_name + '_tracks.h5', obj_type='obj_type_1')
        # print(loc)

        # get the tracks as a python list
        tracks = tracker.tracks#(tracking_updates=TRACKING_UPDATES)

        # get the data in a format for napari
        data, properties, graph = tracker.to_napari(ndim=2)
    
    return(data, properties, graph)

In [ ]:
def track_cells(cellpose_masks, mask_numbers, file):
    '''Run the BTrack algorithm.'''
    
    if np.mean(mask_numbers) > 1: 
        # initialise
        configer_file = 'cell_config.json'
        location = os.getcwd() + '/Config_files/'

        # Create a save folder. 
        save_folder = location + '/Tracks/'
        if os.path.exists(save_folder) == False: 
            os.makedirs(save_folder)

        # Find the tracks for the individual cells in the image stack. 
        cell_tracks, properties, graph = find_Btracks(cellpose_masks, configer_file, location,  save_folder, file)
        # Convert output to a dataframe. 
        tracks_df = pd.DataFrame(cell_tracks, columns = ['Track ID', 'Frame', 'x position', 'y position'])
    else: 
        no_tracks = {'Track ID': [0], 
                     'Frame': [0],
                     'x position': [0], 
                     'y position': [0]
                    }
        tracks_df = pd.DataFrame(no_tracks)
        cell_tracks = []
    
    return(tracks_df, cell_tracks)

--- 
### Rename the cell masks with the track ID instead. 

In [ ]:
def rename_cell_masks(filtered_tracks, mask_array):
    '''Need to rename the cell names with the cell ID to make 
    tracking cells that interact with one another significantly
    easier.'''
    
    # Create a new stack variable for the renamed masks. 
    renamed_masks = np.zeros(mask_array.shape)
    
    if len(filtered_tracks) != 0: 
    
        # Initalised variables. 
        x = filtered_tracks['x position']
        y = filtered_tracks['y position']

        # Find the limits in x and y for the 
        # Cellpose mask frame. 
        x_lims = [0, mask_array.shape[1]]
        y_lims = [0, mask_array.shape[2]]

        # Find the largest mask value in the image. 
        max_label = np.max(filtered_tracks['Track ID']) + 1

        for i in range( int( np.max(filtered_tracks['Track ID']) ) ): 
            # Select track ID. 
            track = i + 1
            track_num_index = np.where(filtered_tracks['Track ID'] == track)[0]

            if len(track_num_index) > 0: 
                # For all the track IDs in the dataframe. 
                for j in range( len(track_num_index) ):
                    # Select the image frame for the track position. 
                    frame = int(filtered_tracks['Frame'].iloc[track_num_index[j]])

                    # Select a single frame from the mask stack. 
                    single_mask_frame = np.array(mask_array[frame, ... ])
                    # Extract the central x and y 
                    # location for the mask in the frame. 
                    x_val = int(x.iloc[track_num_index[j]])
                    y_val = int(y.iloc[track_num_index[j]])

                    # If the mask central position lies outside the 
                    # image boundaries (can happen) reset the 
                    # x/y value to be within the image. 
                    if x_val > x_lims[1]: 
                        x_val = x_lims[1]-2
                    if x_val< x_lims[0]:
                        x_val = x_lims[0] + 2
                    if y_val > y_lims[1]: 
                        y_val = y_lims[1] - 2
                    if y_val < y_lims[0]:
                        y_val = y_lims[0] + 2

                    # Extract the pixel value from the mask that 
                    # corresponds to the xy co-ordinate above. 
                    try: 
                        pixel_val = single_mask_frame[x_val, y_val]
                    except: 
                        # If this is not possible, tell me where the problem is. 
                        print(frame, i, j, x_val, y_val, track_num_index)

                    # If the pixel value is non-zero, create a input into the
                    # new mask variable (rename_mask) with the value of the 
                    # track ID. 
                    if pixel_val == 0: 
                        continue
                    else:
                        single_mask_frame[single_mask_frame != pixel_val] = 0

                        single_mask_frame = ( single_mask_frame/np.max(single_mask_frame) ) * track

                        renamed_masks[frame, :, :] = renamed_masks[[frame], :, :] + (single_mask_frame + max_label )

        renamed_masks = np.array(renamed_masks, dtype = int)

        # Subtract any accumulated background (Might be legacy code). 
        for k in range(renamed_masks.shape[0]):
            renamed_masks[k, ...] = renamed_masks[k, ...] - int( np.median(renamed_masks[k, ...]) )
            
        return(renamed_masks)
        
    else: 
        return(np.array(renamed_masks, dtype = int))        

--- 
### Find the index in the dataframe where the Track ID changes. 

In [ ]:
def get_cell_index(tracks_df, m):
    '''Find the indicies of the tracks dataframe in which the 
    track identifier changes. '''
    
    # Initalise.
    index = []
    track_id = 0
    
    headers = tracks_df.columns.values
    
    # For all values in the tracks dataframe, find 
    # the index where the track ID changes. 
    for i in range( tracks_df.shape[0] ): 
        
        if tracks_df[headers[m]].iloc[i] != track_id: 
            index.append(i)
            track_id = tracks_df[headers[m]].iloc[i]
    
    if len(index) == 1: 
        index.append(i)
    
    return(index)

--- 
### Calculate the distance travelled by the cells for full track length. 

Distance: The total distance a cell travels betweem from i and i+1. 

In [ ]:
def cacluate_movement(index, tracks_df): 
    '''Cacluate how far cells have moved  '''
    
    total_distance_travelled = []

    for i in range( len(index) - 1): 
        # Calculate the number of frames cell tracked.  
        frames_tracked = index[i+1] - index[i]
        # For all the frames tracked. 
        movement = []
        for j in range(frames_tracked-1):
            step_index = index[i] + j
            # Calculate the distance cell travels between frames in x and y. 
            x_change = np.power(tracks_df['x position'].iloc[step_index + 1] - tracks_df['x position'].iloc[step_index], 2)
            y_change = np.power(tracks_df['y position'].iloc[step_index + 1] - tracks_df['y position'].iloc[step_index], 2)
            # Calculate the total distance travelled between frames. 
            distance_travelled = np.sqrt(x_change + y_change)
            # Store all movement steps from macrophages. 
            movement.append(distance_travelled)
        # Cacluate the total distance travelled from start to finish. 
        total_distance_travelled.append( np.sum(movement) )
    
    return(total_distance_travelled)

In [ ]:
def filter_tracks_by_duration(tracks_df, travel_limit, frame_time, pixel_size):
    ''' Create a dataframe including the distance each cell has travelled and 
    for how many frames it was tracked for the inputted cell track dataframe.'''

    # If there are no cells in the tracks_df. 
    if tracks_df['Track ID'].iloc[0] == 0:
    
        filtered_tracks_df = tracks_df
        cell_travel_dict = {'Cell ID': [0], 
                               'Distance Travelled (Pixels)': [0], 
                               'Total Frames Tracked': [0] }

        cell_travel_df = pd.DataFrame(cell_travel_dict)
    
    else:
     
        # Find the index positon of all the phase cells tracked. 
        cell_index = get_cell_index(tracks_df, 0)
           
        # Initialise variables. 
        cells = []
        cell_id_store = []
        filtered_tracks_df = pd.DataFrame()

        for j in range( len(cell_index)- 1 ):

            # If phase cells masks are tracked and move further than 
            # the Maximum distance the green (macrophage) cells. Then add 
            # the cell data to a new dataframe. 

            tracked_frames = cell_index[j + 1] - cell_index[j]

            if tracked_frames > travel_limit:
                
                # Get the cell/track ID.  
                cell_id = tracks_df['Track ID'].iloc[cell_index[j]]
                # Save the cell name and index position. 
                cells.append(cell_id)
                cell_id_store.append(cell_index[j])
                # Add all the cell track data to the filtered dataframe. 
                filtered_tracks_df = pd.concat([filtered_tracks_df, 
                                                     tracks_df[tracks_df['Track ID'] == cell_id]], ignore_index= True)
        
        
        
        # Find all the indicies in the new dataframe where 
        # track/cell id changes. 
        filtered_cells_index = get_cell_index(filtered_tracks_df, 0)   
        
        # Calculate the total movement of the cells that made the cut. 
        cell_travel = cacluate_movement(filtered_cells_index, filtered_tracks_df)

        tracked_frames = []
        
        if len(filtered_cells_index) == 0 or filtered_cells_index[0] == -1: 
           
            cell_travel_dict = {'Cell ID': [0], 
                               'Distance Travelled (Pixels)': [0], 
                               'Total Frames Tracked': [0] }

        else: 
            # print(filtered_cells_index)
            for i in range( len(filtered_cells_index)-1):
                tracked_frames.append( filtered_cells_index[i+1] - filtered_cells_index[i] )

            # Get cell/track IDs. 
            filtered_ids = filtered_tracks_df['Track ID'].iloc[filtered_cells_index[0:-1]]

            cell_travel_dict = {'Cell ID': filtered_ids, 
                               'Distance Travelled (Pixels)': cell_travel, 
                               'Distance Travelled (Microns)': np.array(cell_travel) * pixel_size,
                               'Total Frames Tracked': tracked_frames, 
                               'Time Tracked (Minutes)': np.array(tracked_frames) * frame_time, 
                               'Cell Speed (Microns per minute)': (np.array(cell_travel) * pixel_size) / 
                                                                    (np.array(tracked_frames) * frame_time) }
                
        cell_travel_df = pd.DataFrame(cell_travel_dict)

    return(filtered_tracks_df, cell_travel_df)

In [ ]:
def count_objects(im): 
    '''Count the number of masks in the stack image.  '''

    # Initalise
    object_number = []

    for i in range(im.shape[0]):
        # Get mask indicies. 
        mask_indicies = np.where(im[i, ...] != 0)
        # Initalise
        mask_number = []
        # For all non-zero pixel values. 
        for j in range( len(mask_indicies[0]) ):
            if len( np.where( mask_number == im[i, mask_indicies[0][j], mask_indicies[1][j] ])[0]) == 0: 
                mask_number.append(im[i, mask_indicies[0][j], mask_indicies[1][j]])

        object_number.append( len(mask_number) )
        
    return(np.round((np.mean(object_number)),0))


----
### Create the contact frame

In [ ]:
def create_contact_frame(masks_stack, x_limits, y_limits, frame): 
    '''This is a method that aims to create a layer of indicators for 
    when contacts happen between cells in the image stack. For each 
    time the cells touch each other, a square, centred around the contact 
    is created and added to an othewise empty frame. '''
    
    #initalise
    contact_track_frame = np.zeros([masks_stack.shape[1], masks_stack.shape[2]])
    
    # Get array of contact pixles. 
    x_contact_array = np.arange(x_limits[0], x_limits[1], 1)
    y_contact_array = np.arange(y_limits[0], y_limits[1], 1)
    
    ## Create the contact map. 
    contact_pixles_x = np.where(masks_stack[frame, x_limits[0]:x_limits[1], y_limits[0]:y_limits[1]] > 0)[0]
    contact_pixles_y =  np.where(masks_stack[frame, x_limits[0]:x_limits[1], y_limits[0]:y_limits[1]] > 0)[1]
    
    if len(contact_pixles_x) != 0: 
        # Get mean co-ordinate index for contact
        mean_x_index = int(np.mean(contact_pixles_x))
        mean_y_index = int(np.mean(contact_pixles_y))

        # Find the mean contact position. 
        mean_x = x_contact_array[mean_x_index]
        mean_y = y_contact_array[mean_y_index]

        # Create the box arrays that will go around the contact. 
        box_array_x = np.arange(mean_x-5, mean_x+5, 1)
        box_array_y = np.arange(mean_y-5, mean_y+5, 1)

        # Create a box around the cell. 
        box = [np.array([[box_array_x[0]]*10, box_array_y]), 
              np.array([[box_array_x[-1]]*10, box_array_y]), 
              np.array([box_array_x, [box_array_y[0]]*10]), 
              np.array([box_array_x, [box_array_y[-1]]*10])]

        # Set the square around the cell in the frame. 
        for m in range( len(box)):    
            contact_track_frame[box[m][0], box[m][1]] = 1
    
    return(contact_track_frame)

-----
### Count cells masks

In [ ]:
def count_cell_masks(mask_stack): 
    '''The method takes a cell mask stack and finds the number of 
    masks in each frame. If there are no masks present, then it returns a zero. '''
    
    # Initalise 
    masks_in_frame = []
    
    for i in range(mask_stack.shape[0]):
        # Get the cell masks. 
        cell_mask_indicies = np.where(mask_stack[i, ... ] != 0 )
        
        # Initalise
        mask_values = []
        
        # Loop around the cell mask indicies. 
        for j in range( len(cell_mask_indicies[0]) ):
            # If the mask value has not already been saved, add to variable. 
            if len( np.where(mask_values == mask_stack[i, cell_mask_indicies[0][j], cell_mask_indicies[1][j]])[0] ) == 0:
                mask_values.append(mask_stack[i, cell_mask_indicies[0][j], cell_mask_indicies[1][j]]) 
                
        masks_in_frame.append(len(mask_values))
    
    return(masks_in_frame)

---- 
### Catalogue touching cells

In [ ]:
def find_cells_in_contact(tracks_df, masks_stack_1, masks_stack_2):
    '''Search through all the frames in the mask stack, and a 
    fluorescence image, and look to see if there is overlap between a 
    square with +2 pixels around the cell mask (mask_stack_1) and 
    the fluorescence image masks (mask_stack_2).  
    '''
    # Initalise
    contact_tracker = pd.DataFrame()
    image_lim_x = masks_stack_1.shape[1] - 1
    image_lim_y = masks_stack_1.shape[2] - 1
    
    interaction_map = np.zeros(masks_stack_1.shape)

    cell_indicies_1 = count_cell_masks(masks_stack_1)
    cell_indicies_2 = count_cell_masks(masks_stack_2)
    
    
    if np.mean(cell_indicies_1) < 1 or np.mean(cell_indicies_2) < 1 or len(tracks_df) == 0: 
        temp_dict = {'Gamma Delta T cell': [0], 
                                'Marcophage': [0], 
                                 'Frame': [0]}

        contact_tracker = pd.concat([contact_tracker, pd.DataFrame(temp_dict)])
    
    else: 
        # For all frames in the image. 
        for i in range(masks_stack_1.shape[0]):
            # Find the tracking data for a given frame. 
            existing_cells = tracks_df[tracks_df['Frame'] == i]

            # For all of the tracked cells in the phase channel
            # in frame i.
            for j in range(existing_cells.shape[0]): 
                # Get the cell ID. 
                singe_cell_data = existing_cells.iloc[j]

                # Find all of the pixels corresponding to this cell mask value. 
                location = np.where(masks_stack_1[i, ...] == int(singe_cell_data['Track ID']))
                # If cell doesn't exist (missing segmentation mask, but still included in tracking)
                # Then skip to next cell. 
                if len(location[0]) == 0: 
                    continue
                else:
                    # Find the maximum and minimum x and y values around the cell (a box around the cell). 
                    x_lims = [int(np.min(location[0])), int(np.max(location[0]))]
                    # Check the min and max exist within the image pixel boundaries. 
                    if x_lims[0] < 0: 
                        x_lims[0] = 0 
                    if x_lims[1] > image_lim_x: 
                        x_lims[1] = image_lim_x

                    y_lims = [int(np.min(location[1])), int(np.max(location[1]))]
                    # Check the min and max exist within the image pixel boundaries. 
                    if y_lims[0] < 0: 
                        y_lims[0] = 0 
                    if y_lims[1] > image_lim_y: 
                        y_lims[1] = image_lim_y

                    # If there is the space, create a box around the cell 
                    # Plus 2 pixels. 
                    if x_lims[0] > 1: 
                        x_lims[0] = x_lims[0] - 1
                    if x_lims[1] < image_lim_x - 1 : 
                        x_lims[1] = x_lims[1] + 1
                    if y_lims[0] > 1: 
                        y_lims[0] = y_lims[0] - 1
                    if y_lims[1] <  image_lim_y - 1: 
                        y_lims[1] = y_lims[1] + 1              
  
                    # If the box around the tracked cell in the phase channel touches 
                    # any green cells in the fluorescence channel, then store this as an interaction. 
                    if int(np.min(x_lims)) == int(np.max(x_lims)) or int(np.min(y_lims)) == int(np.max(y_lims)): 
                        continue 
                    else:
                        if np.max(masks_stack_2[i, int(np.min(x_lims)):int(np.max(x_lims)), int(np.min(y_lims)):int(np.max(y_lims))]) > 0: 

                            # Create dictionary for contacts. 
                            temp_dict = {'Gamma Delta T cell': [int(singe_cell_data['Track ID'])], 
                                        'Marcophage': [int(np.max(masks_stack_2[i, int(np.min(x_lims)):int(np.max(x_lims)),
                                                                                int(np.min(y_lims)):int(np.max(y_lims))]))], 
                                         'Frame': [i]}
                            # Turn it into a dataframe. 
                            contact_tracker = pd.concat([contact_tracker, pd.DataFrame(temp_dict)])
                            # Create a contact map for the image stack. 
                            interaction_map[i, ...] = interaction_map[i, ...] + create_contact_frame(masks_stack_2, x_lims, y_lims, i)

            
        if len(contact_tracker) == 0: 
            temp_dict = {'Gamma Delta T cell': [0], 
                                    'Marcophage': [0], 
                                     'Frame': [0]}

            contact_tracker = pd.concat([contact_tracker, pd.DataFrame(temp_dict)])

        else:
            contact_tracker = contact_tracker.sort_values(['Gamma Delta T cell', 'Frame'])
    
    return(contact_tracker, interaction_map)

-----
### Data frame for Number of different cells and interactions per frame. 

In [ ]:
def count_live_cell_interactions(phase_masks_stack, phase_tracks_df, contact_tracker, 
                                green_object, red_object):
    '''Count the number of times that a GDT cell interacts with 
    a live macrophage cell. '''
    
    # initialise. 
    frame_analysis_df = pd.DataFrame()

    if contact_tracker['Gamma Delta T cell'].iloc[0] != 0:
    
        for i in range( phase_masks_stack.shape[0] ):
            # Select frame. 
            frame = i 
            # Filter the objects to include only those that 
            # exist in frame i. 
            num_phase_objects = phase_tracks_df[
                            phase_tracks_df['Frame'] == i]

            # Coun the number of GDT, Macrophage and dead cells. 
            num_gdt_cells = num_phase_objects.shape[0]
            num_macro_cells = green_object[i]
            num_dead_cells = red_object[i]

            # Conut the number of interactions. 
            num_of_contacts = contact_tracker[
                        contact_tracker['Frame'] == i].shape[0]

            # Create a dataframe of the results. 
            frame_anal_dict = {'Frame': [i+1], 
                               'Number of Gamma Delta t cell tracks': [num_gdt_cells], 
                               'Number of Macrophage cells tracks': [num_macro_cells],
                               'Number of Dead cells tracks': [num_dead_cells], 
                               'Number of Gamma Delta t cell - Macrophage interactions': [num_of_contacts] 
                              }

            frame_analysis_df = pd.concat([frame_analysis_df, pd.DataFrame(frame_anal_dict)])
    else: 
        # Create a dataframe of the results. 
            frame_anal_dict = {'Frame': [0], 
                               'Number of Gamma Delta t cell tracks': [0], 
                               'Number of Macrophage cell tracks': [0],
                               'Number of Dead cell tracks': [0], 
                               'Number of Gamma Delta t cell - Macrophage interactions': [0] 
                              }

            frame_analysis_df = pd.concat([frame_analysis_df, pd.DataFrame(frame_anal_dict)])

    return(frame_analysis_df)

----
### Find the number of unique contacts in the image, and the length of interactions

In [ ]:
def count_unique_interactions(contact_tracker, frame_time): 
    '''Count the number of unique interactions between GDT cells and 
    Macrophages. '''
    
    # initalise. 
    unique_contacts = 0
    contact_cells = pd.DataFrame()
    
    # For all interactions. 
    for i in range(contact_tracker.shape[0]): 
        
        # If i == 0. 
        if len(contact_cells) == 0:
            # If there are not cells in contact. 
            if contact_tracker['Marcophage'].iloc[0] == 0: 
                contact_cells = pd.concat([contact_cells, pd.DataFrame({
                                'gdt Cell': [0], 
                                'Macrophage cell': [0], 
                                'Number of frames in contact': [0],
                                'Time in contact (Minutes)': [0]})],)
            else:
                # Populate the dataframe with the first values. 
                contact_cells = pd.concat([contact_cells, pd.DataFrame({
                                'gdt Cell': [contact_tracker['Gamma Delta T cell'].iloc[0]], 
                                'Macrophage cell': [contact_tracker['Marcophage'].iloc[0]], 
                                'Number of frames in contact': [1]})])
        
        else:
            # If it's a new set of interacting cells. 
            if len(np.where(contact_cells['gdt Cell'] == contact_tracker['Gamma Delta T cell'].iloc[i])[0]) == 0 or len(
                    np.where(contact_cells['Macrophage cell'] == contact_tracker['Marcophage'].iloc[i])[0]) == 0: 
                
                # Add them to the data frame. 
                contact_cells = pd.concat([contact_cells, pd.DataFrame({
                            'gdt Cell': [contact_tracker['Gamma Delta T cell'].iloc[i]], 
                            'Macrophage cell': [contact_tracker['Marcophage'].iloc[i]], 
                            'Number of frames in contact': [1]})])
                # Update the number of unique interactions. 
                unique_contacts = unique_contacts + 1

            else: 
                # If it's not a new interaction, update the number of 
                # frames that the cells are in contact. 
                contact_cells['Number of frames in contact'].iloc[unique_contacts
                                        ] = contact_cells['Number of frames in contact'].iloc[unique_contacts] + 1
        
    contact_cells['Time in contact (Minutes)'] = contact_cells['Number of frames in contact'] * frame_time

    # Initialise
    interacting_gdt_cell = []
    interacting_macro_cells = []
    
    # Count how many GDT and Macrophages have an interaction. 
    for j in range(contact_cells.shape[0]): 
        if len(np.where(interacting_gdt_cell == contact_cells['gdt Cell'].iloc[j])[0]) == 0:
            interacting_gdt_cell.append(contact_cells['gdt Cell'].iloc[j])
        if len(np.where(interacting_macro_cells == contact_cells['Macrophage cell'].iloc[j])[0]) == 0:
            interacting_macro_cells.append(contact_cells['Macrophage cell'].iloc[j])

    return(contact_cells, unique_contacts, interacting_gdt_cell, interacting_macro_cells)

------------
### Create Dataframes for Unique and Non-Unique cell interactions per cell track. 

In [ ]:
def get_non_unique_interactions(track_dataframe, interaction_df, n, m , frame_time):
    '''Count the number of times that GDT Cell A interacts with Macrophage B. 
    These include interactions in which Cell A leaves Cell B for N-frames, and then 
    returns. '''

    # initalise
    cell_track_interactions_NonUnique = pd.DataFrame()
    gdt_cell_index = get_cell_index(track_dataframe, 0)

    # Get the headers from the interaction dataframe. 
    headers = interaction_df.columns.values

    # If there are no interactions (eg: controls)
    if len(gdt_cell_index) == 0:
        # Create an empty entry into the dataframe.
        interaction_dic = {'Track ID': [], 
                               'Number of interactions': [], 
                               'Mean Length of interaction (Minutes)': [], 
                               'Standard Deviation Length of interaction (Minutes)': []}

        cell_track_interactions_NonUnique = pd.concat([cell_track_interactions_NonUnique, 
                                                     pd.DataFrame(interaction_dic)])

        zero_interactions = cell_track_interactions_NonUnique[
                            cell_track_interactions_NonUnique['Number of interactions'] == 0 ]

    # For tracked cells 
    for i in range( len(gdt_cell_index) ):
        # Get track ID. 
        track_id = track_dataframe['Track ID'].iloc[gdt_cell_index[i]]
        # Count the number of interactions
        interactions = np.where(interaction_df[headers[n]] == track_id)[0]
        number_of_interactions = len(interactions)

        # I there are no interactions, add in a zero row to the dataframe. 
        if number_of_interactions == 0: 
            interaction_dic = {'Track ID': [track_id], 
                               'Number of interactions': [0], 
                               'Mean Length of interaction (Minutes)': [0 * frame_time], 
                               'Standard Deviation Length of interaction (Minutes)': [0 * frame_time]}

            cell_track_interactions_NonUnique = pd.concat([cell_track_interactions_NonUnique, 
                                                         pd.DataFrame(interaction_dic)])
            
            zero_interactions = cell_track_interactions_NonUnique[
                                cell_track_interactions_NonUnique['Number of interactions'] == 0 ]

        # If there are interacting cells. 
        else: 
            # Initalise. 
            length_of_interaction = 0
            interaction_length_store = []
            first_id = interaction_df[headers[m]].iloc[interactions[0]]
            # For the numebr of interactions. 
            for j in range(number_of_interactions): 
                # If the interaction between these two cells has previously been identified, 
                #  add 1 to the total number of interactions. 
                if interaction_df[headers[m]].iloc[interactions[j]] == first_id: 
                    length_of_interaction = length_of_interaction + 1
                # If it is a new interaction. 
                else: 
                    # Initalise 
                    interaction_length_store.append(length_of_interaction)
                    length_of_interaction = 1
                    first_id = interaction_df[headers[m]].iloc[interactions[j]]
            # Register the length of the interaction between the two cells. 
            interaction_length_store.append(length_of_interaction)
            # Update the interaction dataframe. 
            interaction_dic = {'Track ID': [track_id], 
                               'Number of interactions': [number_of_interactions], 
                               'Mean Length of interaction (Minutes)': [np.mean(interaction_length_store) * frame_time], 
                               'Standard Deviation Length of interaction (Minutes)': [np.std(interaction_length_store) * frame_time]}

            cell_track_interactions_NonUnique = pd.concat([cell_track_interactions_NonUnique, 
                                                         pd.DataFrame(interaction_dic)])
            # Include the cells that have no interactions in a separate dataframe. 
            zero_interactions = cell_track_interactions_NonUnique[
                                cell_track_interactions_NonUnique['Number of interactions'] == 0 ]

    return(cell_track_interactions_NonUnique, zero_interactions)

In [ ]:
def count_unique_interactions_per_track(unique_interaction_df, frame_time): 
    '''Count the number of unique interactions between Cell A and Cell B from the above method
    description.'''

    # Initalise. 
    gdt_index = get_cell_index(unique_interaction_df, 0)
    macrophage_index = get_cell_index(unique_interaction_df, 1)
    unique_interacts_gdt = pd.DataFrame()
    unique_interacts_macro = pd.DataFrame()

    ######
    # Analysis of GDT cell interactions.
    ######
    
    # If there are no interactions, populate with zeros.     
    if len(gdt_index) == 0: 
        unique_interacts_gdt_dict = {'GDT ID': [0],
                                     'Unique Interactions': [0], 
                                     'Number of frames in contact': [0], 
                                     'Total Contact time (Minutes)': [0]}

        unique_interacts_gdt = pd.concat([unique_interacts_gdt, 
                                          pd.DataFrame(unique_interacts_gdt_dict)])
    # If there are interactions. 
    else: 
        # For all the interacting GDT cells. 
        for i in range( len(gdt_index)-1 ): 
            # Build a dataframe of all the unique cell interactions. 
            unique_interacts_gdt_dict = {'GDT ID': [unique_interaction_df['gdt Cell'].iloc[gdt_index[i]]],
                                         'Unique Interactions': [gdt_index[i+1] - gdt_index[i]], 
                                         'Number of frames in contact': [unique_interaction_df['Number of frames in contact'].iloc[gdt_index[i]:gdt_index[i+1]].sum()], 
                                     'Total Contact time (Minutes)': [unique_interaction_df['Number of frames in contact'].iloc[gdt_index[i]:gdt_index[i+1]].sum() * frame_time]}

            unique_interacts_gdt = pd.concat([unique_interacts_gdt, 
                                              pd.DataFrame(unique_interacts_gdt_dict)])

        # If the GDT cell in question only has 1 frame of interaction. 
        # The above method does not like this eventuality. 
        if unique_interaction_df['gdt Cell'].iloc[gdt_index[i+1]] !=  unique_interaction_df['gdt Cell'].iloc[gdt_index[i]]: 
            unique_interacts_gdt_dict = {'GDT ID': [unique_interaction_df['gdt Cell'].iloc[gdt_index[i+1]]],
                                         'Unique Interactions': [1], 
                                         'Number of frames in contact': [1], 
                                     'Total Contact time (Minutes)': [frame_time]}

            unique_interacts_gdt = pd.concat([unique_interacts_gdt, 
                                              pd.DataFrame(unique_interacts_gdt_dict)])

        # IF there is more than 1 interaction between the cells. Add the interaction to the 
        # dataframe. 
        ### NOTE: I feel like this should result in doubling the output in the file produced here, but 
        ### I have checked the output and this is not the case. Which I find a little confusing. 
        else: 
            unique_interacts_gdt_dict = {'GDT ID': [unique_interaction_df['gdt Cell'].iloc[gdt_index[i]]],
                                         'Unique Interactions': [gdt_index[i+1] - gdt_index[i]], 
                                         'Number of frames in contact': [unique_interaction_df['Number of frames in contact'].iloc[gdt_index[i]:gdt_index[i+1]].sum()], 
                                     'Total Contact time (Minutes)': [unique_interaction_df['Number of frames in contact'].iloc[gdt_index[i]:gdt_index[i+1]].sum() * frame_time]}

            unique_interacts_gdt = pd.concat([unique_interacts_gdt, 
                                              pd.DataFrame(unique_interacts_gdt_dict)])

    ######
    # Analysis of Maacorphage cell interactions.
    ######

    # If there are no interactions, populate with zeros.
    if len(macrophage_index) == 0: 
        unique_interacts_macro_dict = {'Macrophage ID': [0],
                                     'Unique Interactions': [0], 
                                     'Number of frames in contact': [0], 
                                     'Total Contact time (Minutes)': [0]}

        unique_interacts_macro = pd.concat([unique_interacts_macro, 
                                          pd.DataFrame(unique_interacts_macro_dict)])
    # If there are interactions. 
    else: 
        # initialisation
        unique_interaction_df = unique_interaction_df.sort_values(['Macrophage cell'])
        macrophage_index = get_cell_index(unique_interaction_df, 1)
        # Build a dataframe of all the unique cell interactions
        for j in range( len(macrophage_index)-1 ): 
            unique_interacts_macro_dict = {'Macrophage ID': [unique_interaction_df['Macrophage cell'].iloc[macrophage_index[j]]],
                                         'Unique Interactions': [macrophage_index[j+1] - macrophage_index[j]], 
                                         'Number of frames in contact': [unique_interaction_df['Number of frames in contact'].iloc[macrophage_index[j]:macrophage_index[j+1]].sum()], 
                                     'Total Contact time (Minutes)': [unique_interaction_df['Number of frames in contact'].iloc[macrophage_index[j]:macrophage_index[j+1]].sum() * frame_time]}

            unique_interacts_macro = pd.concat([unique_interacts_macro, 
                                              pd.DataFrame(unique_interacts_macro_dict)])
        # If the Macrophage cell in question only has 1 frame of interaction. 
        #  The above method does not like this eventuality. 
        if unique_interaction_df['Macrophage cell'].iloc[macrophage_index[j+1]] !=  unique_interaction_df['Macrophage cell'].iloc[macrophage_index[j]]: 
            unique_interacts_macro_dict = {'Macrophage ID': [unique_interaction_df['Macrophage cell'].iloc[macrophage_index[j+1]]],
                                         'Unique Interactions': [1], 
                                         'Number of frames in contact': [1], 
                                         'Total Contact time (Minutes)': [frame_time]}

            unique_interacts_macro = pd.concat([unique_interacts_macro, 
                                              pd.DataFrame(unique_interacts_macro_dict)])
        # If there is more than 1 interaction between the two cells. 
        else: 
            unique_interacts_macro_dict = {'Macrophage ID': [unique_interaction_df['Macrophage cell'].iloc[macrophage_index[j]]],
                                         'Unique Interactions': [macrophage_index[j+1] - macrophage_index[j]], 
                                         'Number of frames in contact': [unique_interaction_df['Number of frames in contact'].iloc[macrophage_index[j]:macrophage_index[j+1]].sum()], 
                                     'Total Contact time (Minutes)': [unique_interaction_df['Number of frames in contact'].iloc[macrophage_index[j]:macrophage_index[j+1]].sum() * frame_time]}

            unique_interacts_macro = pd.concat([unique_interacts_macro, 
                                              pd.DataFrame(unique_interacts_macro_dict)])

    return(unique_interacts_gdt, unique_interacts_macro)

In [ ]:
def create_summary(mean_macrophage_cells, mean_dead_cells, mean_number_gdt_cells , phase_cell_travel_df, 
                  green_cell_travel_df, red_cell_travel_df, total_experiment_time, 
                  renamed_phase_masks, unique_interactions, contact_cells, 
                  images_folder, interacting_gdt, interacting_macro): 
    
    '''Create an analysis summary of the well data.'''
    
    # Calcuate the mean distance travelled by all cells segmented in...
    # The phase channel.
    mean_gdt_cell_distance = phase_cell_travel_df['Distance Travelled (Pixels)'].mean()
    std_gdt_cell_distance = phase_cell_travel_df['Distance Travelled (Pixels)'].std()
    # The Green Channel
    mean_macro_cell_distance = green_cell_travel_df['Distance Travelled (Pixels)'].mean()
    std_macro_cell_distance = green_cell_travel_df['Distance Travelled (Pixels)'].std()
    # The Red Channel 
    mean_dead_cell_distance = red_cell_travel_df['Distance Travelled (Pixels)'].mean()
    std_dead_cell_distance = red_cell_travel_df['Distance Travelled (Pixels)'].std()
    
    # Calculate the mean distance travelled per frame for: 
    # Phase channel. 
    mean_phase_cell_dist_per_frame = np.mean( phase_cell_travel_df['Distance Travelled (Pixels)'] / 
                                       phase_cell_travel_df['Total Frames Tracked'] )
    # Green Channel
    mean_green_cell_dist_per_frame = np.mean( green_cell_travel_df['Distance Travelled (Pixels)'] / 
                                       green_cell_travel_df['Total Frames Tracked'] )
    # Red Channel
    mean_red_cell_dist_per_frame = np.mean( red_cell_travel_df['Distance Travelled (Pixels)'] / 
                                       red_cell_travel_df['Total Frames Tracked'] )
    
    # Calculate time between image frame. 
    time_between_frames = total_experiment_time / renamed_phase_masks.shape[0] 

    # Calculate the mean cell speed per frame for... 
    # The Phase Channel. 
    mean_phase_cell_speed = mean_phase_cell_dist_per_frame / time_between_frames
    std_phase_cell_speed = np.std( phase_cell_travel_df['Distance Travelled (Pixels)'] / 
                                       phase_cell_travel_df['Total Frames Tracked'] ) / time_between_frames
    # The Green Channel 
    mean_green_cell_speed = mean_green_cell_dist_per_frame / time_between_frames
    std_green_cell_speed = np.std( green_cell_travel_df['Distance Travelled (Pixels)'] / 
                                       green_cell_travel_df['Total Frames Tracked'] ) / time_between_frames
    # The Red Channel
    mean_red_cell_speed = mean_red_cell_dist_per_frame / time_between_frames
    std_red_cell_speed = np.std( red_cell_travel_df['Distance Travelled (Pixels)'] / 
                                       red_cell_travel_df['Total Frames Tracked'] ) / time_between_frames

    # Calculate the mean length of interaction between frames. 
    mean_length_of_interaction = contact_cells['Number of frames in contact'].mean()
    std_length_of_interaction = contact_cells['Number of frames in contact'].std()

    ##### 
    # Count interacting cells: 
    num_gdt = int(mean_number_gdt_cells)
    num_macro = int(mean_macrophage_cells)
    num_dead = int(mean_dead_cells)
    
    # Count cell tracks 
    gdt_tracks = phase_cell_travel_df.shape[0]
    macro_tracks = green_cell_travel_df.shape[0]
    dead_tracks = red_cell_travel_df.shape[0]
    
    # Ratio Calc 
    try:
        gdt_ratio =  gdt_tracks / num_gdt
    except: 
        gdt_ratio = 'NAN'
    try:
        macro_ratio =  macro_tracks / num_macro
    except: 
        macro_ratio = 'NAN'
    try:
        dead_ratio =  dead_tracks / num_dead
    except: 
        dead_ratio = 'NAN'

    # Calcuate the percentage of interacting cell tracks. 
    if interacting_gdt[0] == 0:
        percent_gdt_interact = 0 
        percent_macro_interact = 0
        interacting_gdt = []
        interacting_macro = []
        
    else:  
        percent_gdt_interact = 100 * (len(interacting_gdt_cell) / gdt_tracks)
        percent_macro_interact = 100 * (len(interacting_macro_cells) / macro_tracks)

    # Get identifier values.  
    experiment_name = os.path.basename(
                    os.path.dirname(
                    os.path.dirname(
                    os.path.dirname(
                    os.path.dirname(images_folder) ) ) ) )

    well = os.path.basename(os.path.dirname(
                    images_folder))
    

    # Create the summary data frame. 
    image_summary_dataframe_dict = {'Experiment': [experiment_name], 
                                    'Well': [well], 
                                    
                                    'Number of Gamma Delta T cells': [num_gdt], 
                                    'Number of GDT Cell Tracks': [gdt_tracks], 
                                    'GDT Tracks vs Cells Ratio': [gdt_ratio], 
                                    'Mean GDT Cell Distance Tracked (microns)': [mean_gdt_cell_distance * image_resolution], 
                                    'Standard deviation in GDT cell distance tracked (microns)': [std_gdt_cell_distance * image_resolution], 
                                    'Mean GDT Cell speed per frame (microns/minute)': [mean_phase_cell_speed * image_resolution],
                                    'Standard deviation of GDT cells per frame (microns/minute)': [std_phase_cell_speed * image_resolution],
                                    
                                    'Number of Macrophage cells': [num_macro], 
                                    'Number of Macrophage Tracks': [macro_tracks], 
                                    'Macrophage Tracks vs Cells Ratio': [macro_ratio],
                                    'Mean Macrophage Cell Distance Tracked (microns)': [mean_macro_cell_distance * image_resolution], 
                                    'Standard deviation in Macrophage cell distance tracked (microns)': [std_macro_cell_distance * image_resolution],
                                    'Mean Macrophage Cell speed per frame (microns/minute)': [mean_green_cell_speed * image_resolution],
                                    'Standard deviation of Macrophage cells per frame (microns/minute)': [std_green_cell_speed * image_resolution],
                                    
                                    'Number of Dead Cells': [num_dead], 
                                    'Number of Dead Cell Tracks': [dead_tracks], 
                                    'Dead Cell Tracks vs Cells Ratio': [dead_ratio], 
                                    'Mean Dead Cell Distance Tracked (microns)': [mean_dead_cell_distance * image_resolution], 
                                    'Standard deviation in Dead cell distance tracked (microns)': [std_dead_cell_distance * image_resolution],
                                    'Mean Dead Cell speed per frame (microns/minute)': [mean_red_cell_speed * image_resolution],
                                    'Standard deviation of Dead cells per frame (microns/minute)': [std_red_cell_speed * image_resolution],
                                    
                                    '# of GDT cell tracks interacting with macrophages': [len(interacting_gdt)], 
                                    '# of macrophage tracks interacting with GDT cells': [len(interacting_macro)], 
                                    '% of interacting GDT cells Tracks': [percent_gdt_interact],
                                    '% of interacting Macrophage cells Tracks': [percent_macro_interact],
                                    'Mean Length of interaction (minutes)': [mean_length_of_interaction * time_between_frames], 
                                    'Standard Deviation in cell interaction length': [std_length_of_interaction * time_between_frames]
                                    }


    image_summary_dataframe = pd.DataFrame(image_summary_dataframe_dict)
    
    return(image_summary_dataframe)

--- 
### Save the cell segmentations and cell tracks. 

In [ ]:
def save_data(renamed_green_masks, renamed_phase_masks, renamed_red_masks, interaction_stack, 
             filtered_phase_tracks_df, green_tracks_df, red_tracks_df, 
             summary_df, cell_interaction_df, path, cell_type_df ): 

    save_folder = path + '/segmentation_and_tracking/'

    if os.path.exists(save_folder) == False: 
        os.makedirs(save_folder)
        
    image_mask_folder = save_folder + '/Cell_Segmentations/'
    if os.path.exists(image_mask_folder) == False: 
        os.makedirs(image_mask_folder)

    tf.imwrite(image_mask_folder + 'Macrophage_masks.tif', 
              renamed_green_masks)
    tf.imwrite(image_mask_folder + 'gamma_d_t_cells_masks.tif', 
              renamed_phase_masks)
    tf.imwrite(image_mask_folder + 'dead_cells_masks.tif', 
              renamed_red_masks)
    tf.imwrite(image_mask_folder + 'interaction_mask.tif', 
              interaction_stack)
    
    full_track_loc = save_folder + '/Tracking_data/'
    if os.path.exists(full_track_loc) == False: 
        os.makedirs(full_track_loc)

    filtered_phase_tracks_df.to_csv(full_track_loc + 'gamma_delta_t_cell_tracks.csv', index=False)
    green_tracks_df.to_csv(full_track_loc + 'macrophage_tracks.csv', index=False)
    red_tracks_df.to_csv(full_track_loc + 'dead_cell_tracks.csv', index=False)

    experiment_name = os.path.basename(
                os.path.dirname(
                os.path.dirname(path)))
    
    if os.path.isfile(os.path.dirname( 
                os.path.dirname(path) )  + '/' + experiment_name + 
                      '_well_summary_file.csv') == True: 
        
        summary_df.to_csv(os.path.dirname( 
                os.path.dirname(path) ) + '/' + experiment_name + 
                          '_well_summary_file.csv', mode = 'a', 
                         header = False, index=False)
    else:
        summary_df.to_csv(os.path.dirname( 
                os.path.dirname(path) ) + '/' + experiment_name + 
                          '_well_summary_file.csv', index=False)
    
    cell_interaction_df.to_csv(save_folder + 'Cell_Interactions_Per_Frame.csv', index=False)
    
    cell_type_df.to_csv(save_folder + 'Cell_Type_and_number_tracking.csv', index = False)
    
    return(save_folder)

-----------
### Save the cell Phonetype tracking throughout the movie as a dataframe

In [ ]:
def cell_phenotype_track(gdt_number, macro_number, dead_number):
    '''Record the cell type classifications in a dataframe to aid with analysis/wierd results.'''
    
    
    cell_phenotype_dict = {'Frame': np.arange(1, len(gdt_number)+1, 1), 
                           'Gamma Delta T Cell number': gdt_number, 
                           'Macrophage Number': macro_number, 
                           'Dead Cell Number': dead_number}
    
    cell_phenotype_df = pd.DataFrame(cell_phenotype_dict)
    
    return(cell_phenotype_df)

In [ ]:
def find_existing_analysis(parent_folder, experiment): 
    '''This method looks to see if the analysis file exists, and then 
        extracts the name of the wells that have been examined. '''

    if os.path.isfile(parent_folder   
        + '/' + experiment  +'_well_summary_file.csv') == True: 

        existing_data = pd.read_csv(parent_folder + '/' +
                experiment  +'_well_summary_file.csv')

        examined_wells = existing_data['Well']

    else: 
        examined_wells = []
        
    return(examined_wells)

---
### Full script analysis

In [ ]:
print( # Get identifier values.  
    os.path.basename(
                    os.path.dirname(
                    os.path.dirname(
                    os.path.dirname(
                    os.path.dirname(images_folder) ) ) ) ) )


print(os.path.basename(os.path.dirname(
                    images_folder)) )

In [ ]:
# Find all the sub-folder names with the image data inside.
sub_folders = find_image_folders(images_folder)

if len(sub_folders) == 0: 
    sub_folders = [images_folder]

models_folder = os.getcwd() + '/Cellpose_models/'
phase_seg_model = models_folder + '20240201_gdt_cell_segment_phase_2'
green_seg_model = models_folder + '20240201_marcophage_cell_segment_green'


# # For all the wells imaged in the plate. 
for j in range(len(sub_folders) ):
    # # Get all the image files from the folder selected.
    image_dir_content = os.listdir(sub_folders[j])

    well = os.path.basename(
        os.path.dirname(sub_folders[j] ))
    
    experiment_name = os.path.basename(
        os.path.dirname(os.path.dirname(sub_folders[j] )) )[0:5]
    
    experiment_folder = os.path.basename(
        os.path.dirname(os.path.dirname(sub_folders[j] )) )

    examined_wells = find_existing_analysis(
            os.path.dirname(
            os.path.dirname(sub_folders[j])), experiment_folder)

    if len(np.where(examined_wells == well)[0]) == 0: 

        # # Remove non-images from the file list. 
        for i in range( len(image_dir_content) ): 
            # Read in the image file data.
            if image_dir_content[i].find('C1') != -1:
                red_im = tf.imread(sub_folders[j] + '/' + image_dir_content[i])[0:50, ...]
            elif image_dir_content[i].find('C2') != -1:
                green_im = tf.imread(sub_folders[j] + '/' + image_dir_content[i])[0:50, ...]
            elif image_dir_content[i].find('C3') != -1:
                phase_im = tf.imread(sub_folders[j] + '/' + image_dir_content[i])[0:50, ...]
            else:
                continue
    
        total_experiment_time = time_between_frames * red_im.shape[0]
        
        # Preform image thresholding for the fluorescence images. 
        threshold_green = mean_thresholding(green_im)
        threshold_red, red_masks = otzu_thresholding(red_im)

        # # Use the cellpose library to segment the images.  
        # ## Total time 4 minutes on SKYNET.
        green_masks, green_object_number = run_cellpose(green_seg_model, threshold_green, None)    

        # Total time approx: 30mins on SKYNET.
        phase_masks, phase_object_number = run_cellpose(phase_seg_model, phase_im, None)

        print('Segmentation finished')
        
        # Filter masks by green and red mask  content. 
        # If 20% of a phase mask is covered by a green it is designated as a macrophage mask. 
        # If more than 20% of the phase mask is red, it is designated as a dead mask. 
        filtered_phase_masks, filtered_green_masks, filtered_red_masks = filter_masks_by_overlapping_masks(phase_masks, 
                                                                                                           green_masks, red_masks)
        # Filter the phase masks by size. 
        # Size filter is 1.3 times the mean phase mask size. 
        size_filtered_renamed_phase_masks = filter_masks_by_size(filtered_phase_masks)
        
        # Collect the number of masks in the phase channel. 
        renamed_phase_mask_num = count_cell_masks(size_filtered_renamed_phase_masks)
        red_object_number =  count_cell_masks(filtered_red_masks)
        green_object_number =  count_cell_masks(filtered_green_masks)
        
        # Perform cell tracking on all colour channels. 
        green_tracks_df, green_tracks =  track_cells(filtered_green_masks, green_object_number, well)
        filtered_phase_tracks_df, phase_tracks =  track_cells(size_filtered_renamed_phase_masks, renamed_phase_mask_num, well)
        red_tracks_df, red_tracks =  track_cells(filtered_red_masks, red_object_number, well) # execution time ~1min, SKYNET

        print('Tracking Finished')
        
        # Set a limit of only tracking cells that exist for 90% of the frames capture. 
        limit = int(0.9 * phase_im.shape[0])
        # Filter the tracks by duration larger than 90% of total frames.
        filtered_phase_tracks_df, phase_tracks_dist_df = filter_tracks_by_duration(filtered_phase_tracks_df, limit, 
                                                                                   time_between_frames, image_resolution)

        filtered_green_tracks_df, green_tracks_dist_df = filter_tracks_by_duration(green_tracks_df, limit, 
                                                                                   time_between_frames, image_resolution)
        
        filtered_red_tracks_df, red_tracks_dist_df = filter_tracks_by_duration(red_tracks_df, limit, 
                                                                               time_between_frames, image_resolution)
        
        # Rename all the cellpose segmentations with the 
        # Track ID numbers. 
        renamed_green_masks = rename_cell_masks(filtered_green_tracks_df, filtered_green_masks)
        renamed_phase_masks = rename_cell_masks(filtered_phase_tracks_df, size_filtered_renamed_phase_masks)
        renamed_red_masks = rename_cell_masks(filtered_red_tracks_df, filtered_red_masks) # excution time: 16:30 minutes
        
        print('Finished Renaming')

        # Find and document all of the GDT and Macrophage cell. 
        cell_interaction_df, interaction_stack = find_cells_in_contact(filtered_phase_tracks_df, 
                                                renamed_phase_masks, renamed_green_masks)
        
        print('Found all the cell interactions')

        cell_interactions_per_frame = count_live_cell_interactions(renamed_phase_masks,
                                                                   filtered_phase_tracks_df, 
                                                                   cell_interaction_df, green_object_number, red_object_number)
        
        # Document the unique interactions. 
        unique_interactions, unique_contacts, interacting_gdt_cell, interacting_macro_cells = count_unique_interactions(cell_interaction_df, time_between_frames)
        
        gdt_cell_non_unique_interactions, gdt_zero_interactions = get_non_unique_interactions(filtered_phase_tracks_df, cell_interaction_df, 0, 1, time_between_frames)
        macro_cell_non_unique_interactions, macro_zero_interactions = get_non_unique_interactions(filtered_green_tracks_df, cell_interaction_df, 1, 0, time_between_frames)

        gdt_unique_interactions, macrophage_unique_interactions = count_unique_interactions_per_track(unique_interactions, time_between_frames)
        
        # Change the headers in the zero interaction dataframes. 
        gdt_zeros = np.array(gdt_zero_interactions)
        heads = gdt_unique_interactions.columns.values
        gdt_zeros = pd.DataFrame(gdt_zeros, columns = heads)
        
        macro_zeros = np.array(macro_zero_interactions)
        heads = gdt_unique_interactions.columns.values
        macro_zeros = pd.DataFrame(macro_zeros, columns = heads)
        # Add the zero interactions to the unique interactions. 
        gdt_unique_interactions = pd.concat([gdt_unique_interactions, pd.DataFrame(gdt_zeros)])
        gdt_unique_interactions = pd.concat([gdt_unique_interactions, pd.DataFrame(macro_zeros)])
        
        # Collect the number of masks in the phase channel. 
        renamed_phase_mask_num = count_cell_masks(size_filtered_renamed_phase_masks)
        red_object_number =  count_cell_masks(filtered_red_masks)
        green_object_number = count_cell_masks(filtered_green_masks)

        # Create a summary document for the well.
        summary_df = create_summary(np.round(np.mean(green_object_number),0), np.round(np.mean(red_object_number),0), 
                                   np.round(np.mean(renamed_phase_mask_num, 0)), phase_tracks_dist_df, green_tracks_dist_df, 
                                    red_tracks_dist_df, total_experiment_time, renamed_phase_masks, unique_contacts, 
                                    unique_interactions, sub_folders[j], interacting_gdt_cell, interacting_macro_cells)
        
        # Track how the number and type of cells identified changes with time. 
        cell_numbers = cell_phenotype_track(renamed_phase_mask_num, green_object_number, red_object_number)

        # Save everything!
        save_path = save_data(renamed_green_masks, renamed_phase_masks, renamed_red_masks, interaction_stack,
             filtered_phase_tracks_df, filtered_green_tracks_df, 
          filtered_red_tracks_df, summary_df, cell_interactions_per_frame, sub_folders[j], cell_numbers)

        # Save interaction data per tracks for Histograms. 
        interaction_folder = sub_folders[j] + '/Interaction_analysis/'
        if os.path.exists(interaction_folder) == False: 
            os.makedirs(interaction_folder)
        
        if len(unique_interactions) == 0:
            continue
        else: 
            with pd.ExcelWriter(interaction_folder + '/' + experiment_name + '_' + 
                                well + "_Interaction_file.xlsx") as writer:
   
                # use to_excel function and specify the sheet_name and index 
                # to store the dataframe in specified sheet
                unique_interactions.to_excel(writer, sheet_name="Unique Interactions", index=False)
                gdt_unique_interactions.to_excel(writer, sheet_name="GDT unique interations", index=False)
                gdt_cell_non_unique_interactions.to_excel(writer, sheet_name="GDT non-unique interactions", index=False)
                macrophage_unique_interactions.to_excel(writer, sheet_name="Macrophage unique interations", index=False)
                macro_cell_non_unique_interactions.to_excel(writer, sheet_name="Macrophage non-unique interactions", index=False)
        
        # Save distance data per track for histograms
        distance_folder = sub_folders[j] + '/Distance_analysis/'
        if os.path.exists(distance_folder) == False: 
            os.makedirs(distance_folder)
            
        phase_tracks_dist_df.to_csv(distance_folder + 'GDT_cell_distance_per_track.csv', index = False)
        green_tracks_dist_df.to_csv(distance_folder + 'Macrophage_cell_distance_per_track.csv', index = False)
        red_tracks_dist_df.to_csv(distance_folder + 'Dead_cell_distance_per_track.csv', index = False)

        print('Saved: ' + well)
    else: 
        print('Skipped Well :' + well)

print('Finished')

--- 
### View outputs of analysis in Napari. 

In [ ]:
viewer = napari.Viewer()
# Add images
viewer.add_image(phase_im)
viewer.add_image(green_im)
viewer.add_image(red_im)

# Add Cell segmentaion
viewer.add_labels(renamed_phase_masks)
viewer.add_labels(renamed_red_masks)
viewer.add_labels(renamed_green_masks)

# Add interaction layer. 
viewer.add_labels(interaction_stack.astype(int))

# Add cell tracks. 
viewer.add_tracks(filtered_phase_tracks_df)
viewer.add_tracks(np.array(filtered_green_tracks_df))
viewer.add_tracks(np.array(filtered_red_tracks_df))